## Batch features using Signals

This notebook creates a new feature view using the SDK that will be computed using stream processing.

### Flow of data

```mermaid
flowchart LR
    sp(Snowplow Pipeline)
    stream[/Autogenerated dbt model/]
    signals(Signals)

    sp --> stream
    stream --> signals
```

---

### Define a new feature

This block creates a single feature definition including the logic how it should be calculated (it's filters and aggregation).

The feature calculates the number of add to cart ecommerce events.

In [1]:
from snowplow_signals import (
    Feature,
    FilterCombinator,
    FilterCondition,
)

products_added_to_cart_feature = Feature(
    name="products_added_to_cart",
    dtype="STRING_LIST",
    events=[
        "iglu:com.snowplowanalytics.snowplow.ecommerce/snowplow_ecommerce_action/jsonschema/1-0-2"
    ],
    type="unique_list",
    property="contexts_com_snowplowanalytics_snowplow_ecommerce_product_1[0].name",
    scope="session",
    filter=FilterCombinator(
        combinator="and",
        condition=[
            FilterCondition(
                property="unstruct_event_com_snowplowanalytics_snowplow_ecommerce_snowplow_ecommerce_action_1:type",
                operator="equals",
                value="add_to_cart",
            ),
        ],
    ),
)

/Users/matus/Library/Caches/pypoetry/virtualenvs/signals-notebooks-ISVnSzaa-py3.13/lib/python3.13/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DataSource" shadows an attribute in parent "BaseSignalsObject"
  warnings.warn(


### Wrapping the feature in a feature view

All features need to be included in feature views that can be considered as "tables" of features.

Feature views are immutable and versioned.

In [2]:
from snowplow_signals import FeatureView, user_entity

feature_view = FeatureView(
    name="batch_ecommerce_features",
    version=1,
    entities=[
        user_entity,
    ],
    features=[
        products_added_to_cart_feature,
    ],
)

### Applying the feature view to Signals

The following block pushes the feature view definition to the Signals API and makes it available for processing.

In [92]:
from snowplow_signals import Signals

sp_signals = Signals(api_url='https://0fcfdf97-6447-4208-8cd0-39f82befbd07.svc.snplow.net')
sp_signals.apply([feature_view])

ApplyResponse(status='applied')